<a href="https://colab.research.google.com/github/tanish-garg-1/ML-Lab-Assignments/blob/main/API_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from tsai.all import *
from fastai.tabular.all import *
import warnings

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

def get_models(c_in=2, c_out=1, seq_len=24):
    """Return dictionary of all five models using direct PyTorch implementations."""
    lstm = nn.Sequential(
        nn.LSTM(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )
    gru = nn.Sequential(
        nn.GRU(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )
    mlp = nn.Sequential(
        nn.Flatten(),
        nn.Linear(seq_len * c_in, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, c_out)
    )
    fcn = FCN(c_in=c_in, c_out=c_out)
    resnet = ResNet(c_in=c_in, c_out=c_out)

    return {'LSTM': lstm, 'GRU': gru, 'MLP': mlp, 'FCN': fcn, 'ResNet': resnet}

def model_forward(model, x, model_name):
    """Handle forward pass for different model types."""
    if model_name in ['LSTM', 'GRU']:
        x = x.transpose(1, 2)  # Change to (batch, seq_len, features)
        if isinstance(model, nn.Sequential):
            output, _ = model[0](x)
            return model[1](output[:, -1, :])
    elif model_name == 'MLP':
        return model(x)  # MLP expects flattened input
    else:
        return model(x)  # FCN and ResNet expect (batch, features, seq_len)

def prepare_data(df, seq_length=24, split_ratio=0.8):
    """Prepare data for models with correct dimensions."""
    df = df.sort_values('Time of Call')
    time_diffs = df['Time of Call'].diff().dt.total_seconds().fillna(0)
    time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()

    X, y = [], []
    for i in range(len(df) - seq_length):
        sequence = np.column_stack([time_diffs.iloc[i:i+seq_length], np.ones(seq_length)])
        X.append(sequence)
        y.append([1])  # Predicting the next event

    X = np.array(X, dtype=np.float32).transpose(0, 2, 1)
    y = np.array(y, dtype=np.float32)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    train_size = int(len(X) * split_ratio)
    splits = (list(range(train_size)), list(range(train_size, len(X))))

    tfms = [None, [TSRegression()]]
    dsets = TSDatasets(X, y, splits=splits, tfms=tfms)
    batch_size = min(64, len(X))
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_size=batch_size)

    return dls, df['Time of Call'].iloc[-1]

def train_and_predict(api, df, seq_length=24):
    """Train models for a given API and make predictions."""
    dls, last_timestamp = prepare_data(df, seq_length)
    models = get_models(seq_len=seq_length)
    predictions = {}

    for model_name, model in models.items():
        print(f"Training {model_name} for {api}...")

        class ModelWrapper(nn.Module):
            def __init__(self, base_model, model_name):
                super().__init__()
                self.base_model = base_model
                self.model_name = model_name

            def forward(self, x):
                return model_forward(self.base_model, x, self.model_name)

        wrapped_model = ModelWrapper(model, model_name)
        learn = Learner(dls, wrapped_model, metrics=[rmse])
        learn.fit_one_cycle(10, lr_max=1e-3)

        # Prepare data for prediction
        time_diffs = df['Time of Call'].diff().dt.total_seconds().fillna(0)
        time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()
        last_sequence = np.column_stack([time_diffs.iloc[-seq_length:].values, np.ones(seq_length)])
        last_sequence = torch.tensor(last_sequence.transpose().reshape(1, 2, seq_length), dtype=torch.float32)

        model.eval()
        with torch.no_grad():
            pred = model_forward(model, last_sequence, model_name)
            predictions[model_name] = max(0, float(pred[0] if len(pred.shape) > 1 else pred))

    return predictions, last_timestamp

def main():
    """Main function to process API data and train models."""
    file_path = "API Call Dataset.csv"
    df = pd.read_csv(file_path)

    # Replace incorrect formats and standardize time strings
    df['Time of Call'] = df['Time of Call'].str.replace(r'\.00$', ':00', regex=True)
    df['Time of Call'] = df['Time of Call'].str.replace(r'(\d)\.(\d+)$', r'\1:\2', regex=True)

    try:
        df['Time of Call'] = pd.to_datetime(df['Time of Call'], format='%d-%m-%Y %H:%M')
    except ValueError as e:
        print("Error in datetime conversion:", e)
        print("Ensure your dataset has a consistent time format.")

    api_counts = df['API Code'].value_counts().nlargest(4)
    top_apis = api_counts.index

    all_predictions = {}
    last_timestamps = {}

    for api in top_apis:
        print(f"\nProcessing API: {api}")
        api_df = df[df['API Code'] == api]
        predictions, last_timestamp = train_and_predict(api, api_df)
        all_predictions[api] = predictions
        last_timestamps[api] = last_timestamp

    print("\nPredictions:")
    print(pd.DataFrame(all_predictions).round(2))

    print("\nLast Timestamps:")
    for api, timestamp in last_timestamps.items():
        print(f"{api}: {timestamp}")

if __name__ == "__main__":
    main()


if __name__ == "__main__":
    main()



Processing API: A9
Training LSTM for A9...


epoch,train_loss,valid_loss,_rmse,time
0,0.772920,0.645536,0.803453,00:01
1,0.404173,0.018954,0.137673,00:01
2,0.185973,0.000040,0.006309,00:01
3,0.093257,0.000031,0.005606,00:01
4,0.048730,0.000025,0.004998,00:01
5,0.025991,0.000021,0.004552,00:01
6,0.014013,0.000018,0.004293,00:00
7,0.007600,0.000017,0.004093,00:01
8,0.004137,0.000016,0.003944,00:01
9,0.002258,0.000015,0.003934,00:01


Training GRU for A9...


epoch,train_loss,valid_loss,_rmse,time
0,0.775737,0.462498,0.680072,00:01
1,0.331843,0.010873,0.104274,00:00
2,0.152541,0.000084,0.009179,00:00
3,0.076485,0.000010,0.003168,00:01
4,0.039960,0.000005,0.002321,00:01
5,0.021308,0.000004,0.002046,00:01
6,0.011484,0.000004,0.001893,00:01
7,0.006225,0.000003,0.001832,00:00
8,0.003385,0.000003,0.001812,00:00
9,0.001844,0.000003,0.001811,00:00


Training MLP for A9...


...

Training FCN for A4...


epoch,train_loss,valid_loss,_rmse,time
0,0.511537,0.389402,0.624021,00:03
1,0.204858,0.003269,0.057175,00:02
2,0.095648,0.005371,0.073290,00:01
3,0.049078,0.006941,0.083313,00:01
4,0.026311,0.002186,0.046752,00:01
5,0.014487,0.002282,0.047772,00:01
6,0.008046,0.000539,0.023214,00:02
7,0.004582,0.000780,0.027932,00:02
8,0.002618,0.000663,0.025740,00:01
9,0.001524,0.000706,0.026573,00:02


Training ResNet for A4...


epoch,train_loss,valid_loss,_rmse,time
0,0.396719,0.458739,0.677303,00:03
1,0.149342,0.009150,0.095654,00:04
2,0.069917,0.002592,0.050912,00:03
3,0.035910,0.002240,0.047328,00:03
4,0.019285,0.001718,0.041448,00:03
5,0.010649,0.002651,0.051491,00:04
6,0.005992,0.001676,0.040939,00:03
7,0.003418,0.001505,0.038789,00:03
8,0.001985,0.001566,0.039578,00:04
9,0.001179,0.001471,0.038357,00:03



Predictions:
          A9    A2    A7    A4
LSTM    1.00  1.00  1.00  1.00
GRU     1.00  1.00  1.00  1.00
MLP     1.00  1.06  0.98  1.00
FCN     0.96  0.94  0.99  0.97
ResNet  0.99  0.92  0.99  1.00

Last Timestamps:
A9: 2025-07-06 09:06:00
A2: 2025-07-06 09:45:00
A7: 2025-07-06 07:55:00
A4: 2025-07-06 10:22:00


In [1]:
pip install tsai


# New Section